In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


### Import Library

In [ ]:
#datacleaning
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import spacy

In [ ]:
spacy.cli.download("en_core_web_lg")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
from spacy.tokens import DocBin
from tqdm import tqdm
import spacy

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#data label
import json
from ctypes import alignment
from pandas.core.tools.numeric import annotations
import timeit

### Import Dataset

In [ ]:
# Dataset = pd.read_csv("/content/drive/MyDrive/Tesis/reviews_MOBA#1.csv")
Dataset = pd.read_pickle("/content/drive/MyDrive/Tesis/saved_data/review_scrap_up2.pkl")
Dataset

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,app_name
0,0db5c544-3b32-49c8-ac71-55623182eabf,Oklahoma Hound,https://play-lh.googleusercontent.com/a/AGNmyx...,I like this game alot. Game controls smooth ga...,3,0,4.0.0.6270,2023-02-28 23:52:36,None,NaT,League of Legends: Wild Rift
1,29333e61-701d-4b4d-91cb-0246d796b375,Bryan Bush,https://play-lh.googleusercontent.com/a-/ACB-R...,For casual players: The most fundamental flaw ...,3,79,4.0.0.6270,2023-02-07 03:21:16,None,NaT,League of Legends: Wild Rift
2,dfca10dc-a5e0-400c-bb9e-ca11672e5948,Vincent Agustin,https://play-lh.googleusercontent.com/a-/ACB-R...,This game is better than other mobile games bu...,3,0,4.0.0.6270,2023-03-01 16:45:36,None,NaT,League of Legends: Wild Rift
3,51323dde-4adb-47c9-8536-c0abebd22412,Andre Koh,https://play-lh.googleusercontent.com/a-/ACB-R...,The game is fun and all when you get to play i...,3,342,4.0.0.6270,2023-01-25 11:28:18,None,NaT,League of Legends: Wild Rift
4,15bf3e31-8ef3-4fd1-9f1b-7dd1ecf6c7b5,Lee Stefan,https://play-lh.googleusercontent.com/a/AGNmyx...,Only exclusive to this game in all games acros...,3,47,4.0.0.6270,2023-01-25 10:12:39,None,NaT,League of Legends: Wild Rift
...,...,...,...,...,...,...,...,...,...,...,...
216858,b94be9c6-4545-4594-b6bd-df3447ad0913,Mayank Gala,https://play-lh.googleusercontent.com/a-/ACB-R...,DONT INSTALL MATCHMAKING IS WORST,1,0,1.1.9.0,2020-06-29 08:32:51,None,NaT,Heroes Evolved
217573,9cf9def6-9438-43f8-97c8-9963f99d02c6,Johny Nabam,https://play-lh.googleusercontent.com/a-/ACB-R...,Your matchmaking sucks..,2,0,1.1.9.0,2020-09-16 15:34:42,None,NaT,Heroes Evolved
217590,2ea2939e-c366-495a-af6c-4a1a7c64dc3f,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,THE GAME IS DEAD,1,2,1.1.9.0,2020-04-16 23:27:24,None,NaT,Heroes Evolved
217607,9a80facb-af88-480c-ab1e-740eb2abf462,noctix nefarioux,https://play-lh.googleusercontent.com/a-/ACB-R...,Lost account,1,0,1.1.9.0,2021-09-27 17:58:51,"Hi, please try to create a new account on the ...",2021-09-29 08:15:32,Heroes Evolved


In [ ]:
Dataset.head(2)

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,app_name
0,0db5c544-3b32-49c8-ac71-55623182eabf,Oklahoma Hound,https://play-lh.googleusercontent.com/a/AGNmyx...,I like this game alot. Game controls smooth ga...,3,0,4.0.0.6270,2023-02-28 23:52:36,None,NaT,League of Legends: Wild Rift
1,29333e61-701d-4b4d-91cb-0246d796b375,Bryan Bush,https://play-lh.googleusercontent.com/a-/ACB-R...,For casual players: The most fundamental flaw ...,3,79,4.0.0.6270,2023-02-07 03:21:16,None,NaT,League of Legends: Wild Rift


### Dataset Exploration

#### Data Cleaning

**Data cleaning** - memeriksa dan menghapus missing value dalam corpus, serta menghapus data yang tidak dibutuhkan dalam proses analisis data review

In [ ]:
print(Dataset.shape)
print(Dataset.dtypes)

(180960, 11)
reviewId                        object
userName                        object
userImage                       object
content                         object
score                            int64
thumbsUpCount                    int64
reviewCreatedVersion            object
at                              object
replyContent                    object
repliedAt               datetime64[ns]
app_name                        object
dtype: object


In [ ]:
for col in Dataset.columns:
    pct_missing = np.mean(Dataset[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

reviewId - 0%
userName - 0%
userImage - 0%
content - 0%
score - 0%
thumbsUpCount - 0%
reviewCreatedVersion - 24%
at - 0%
replyContent - 88%
repliedAt - 88%
app_name - 0%


In [ ]:
df = Dataset.drop(['userImage','thumbsUpCount','reviewCreatedVersion','replyContent','repliedAt'], axis=1)


In [ ]:
df

,reviewId,userName,content,score,at,app_name
0,0db5c544-3b32-49c8-ac71-55623182eabf,Oklahoma Hound,I like this game alot. Game controls smooth ga...,3,2023-02-28 23:52:36,League of Legends: Wild Rift
1,29333e61-701d-4b4d-91cb-0246d796b375,Bryan Bush,For casual players: The most fundamental flaw ...,3,2023-02-07 03:21:16,League of Legends: Wild Rift
2,dfca10dc-a5e0-400c-bb9e-ca11672e5948,Vincent Agustin,This game is better than other mobile games bu...,3,2023-03-01 16:45:36,League of Legends: Wild Rift
3,51323dde-4adb-47c9-8536-c0abebd22412,Andre Koh,The game is fun and all when you get to play i...,3,2023-01-25 11:28:18,League of Legends: Wild Rift
4,15bf3e31-8ef3-4fd1-9f1b-7dd1ecf6c7b5,Lee Stefan,Only exclusive to this game in all games acros...,3,2023-01-25 10:12:39,League of Legends: Wild Rift
...,...,...,...,...,...,...
216858,b94be9c6-4545-4594-b6bd-df3447ad0913,Mayank Gala,DONT INSTALL MATCHMAKING IS WORST,1,2020-06-29 08:32:51,Heroes Evolved
217573,9cf9def6-9438-43f8-97c8-9963f99d02c6,Johny Nabam,Your matchmaking sucks..,2,2020-09-16 15:34:42,Heroes Evolved
217590,2ea2939e-c366-495a-af6c-4a1a7c64dc3f,A Google user,THE GAME IS DEAD,1,2020-04-16 23:27:24,Heroes Evolved
217607,9a80facb-af88-480c-ab1e-740eb2abf462,noctix nefarioux,Lost account,1,2021-09-27 17:58:51,Heroes Evolved


###Memeriksa kemungkinan adanya duplikat dalam dataset

Dokumen review opini mobile game bergenre MOBA, berdasarkan 5 MOBA popular, diantaranya : 'LeagueOfLeague', 'Freefire', 'MobileLegends', 'Herostrike', 'Heroesevolved'

In [ ]:
len(df['reviewId'].unique())

180960

Dataset memasuki proses preprocessing agar lebih mudah diolah oleh komputer, column yang digunakan hanya column komentar dan index data

In [ ]:
df_dataset = df.drop(['reviewId'], axis=1)

In [ ]:
print(df_dataset)

                userName                                            content  \
0         Oklahoma Hound  I like this game alot. Game controls smooth ga...   
1             Bryan Bush  For casual players: The most fundamental flaw ...   
2        Vincent Agustin  This game is better than other mobile games bu...   
3              Andre Koh  The game is fun and all when you get to play i...   
4             Lee Stefan  Only exclusive to this game in all games acros...   
...                  ...                                                ...   
216858       Mayank Gala                  DONT INSTALL MATCHMAKING IS WORST   
217573       Johny Nabam                           Your matchmaking sucks..   
217590     A Google user                                   THE GAME IS DEAD   
217607  noctix nefarioux                                       Lost account   
218667     A Google user  Well made..i wish more skin to come.and pls ma...   

        score                   at                 

In [ ]:
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180960 entries, 0 to 218667
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   userName  180960 non-null  object
 1   content   180960 non-null  object
 2   score     180960 non-null  int64 
 3   at        180960 non-null  object
 4   app_name  180960 non-null  object
dtypes: int64(1), object(4)
memory usage: 8.3+ MB


In [ ]:
df_dataset.content.describe()

count                                                180960
unique                                               180960
top       I like this game alot. Game controls smooth ga...
freq                                                      1
Name: content, dtype: object

In [ ]:
df_dataset = df_dataset.reset_index(drop=True)
df_dataset

,userName,content,score,at,app_name
0,Oklahoma Hound,I like this game alot. Game controls smooth ga...,3,2023-02-28 23:52:36,League of Legends: Wild Rift
1,Bryan Bush,For casual players: The most fundamental flaw ...,3,2023-02-07 03:21:16,League of Legends: Wild Rift
2,Vincent Agustin,This game is better than other mobile games bu...,3,2023-03-01 16:45:36,League of Legends: Wild Rift
3,Andre Koh,The game is fun and all when you get to play i...,3,2023-01-25 11:28:18,League of Legends: Wild Rift
4,Lee Stefan,Only exclusive to this game in all games acros...,3,2023-01-25 10:12:39,League of Legends: Wild Rift
...,...,...,...,...,...
180955,Mayank Gala,DONT INSTALL MATCHMAKING IS WORST,1,2020-06-29 08:32:51,Heroes Evolved
180956,Johny Nabam,Your matchmaking sucks..,2,2020-09-16 15:34:42,Heroes Evolved
180957,A Google user,THE GAME IS DEAD,1,2020-04-16 23:27:24,Heroes Evolved
180958,noctix nefarioux,Lost account,1,2021-09-27 17:58:51,Heroes Evolved


In [ ]:
def clean_lower(text_lower):
    text_lower = text_lower.lower() # lowercase text
    return text_lower

# Buat kolom tambahan untuk data description yang telah dicasefolding  
df_dataset['content'] = df_dataset['content'].apply(clean_lower)

Fungsi mengambil data, untuk dilakukan step pelabelan

In [ ]:
# with open('readme#3.txt', 'w') as f:
#     for line in df_review['content_with_lemma']:
#         f.write(line)
#         f.write('\n')

In [ ]:
part_20 = df_dataset['content'].sample(5000)
part_20

163070            this game is awesome. i need say no more.
29945     i think the game is pretty good but it would b...
73445                           this game is too much laggy
93712     free fire is the best game in my life..but i f...
100516    the service are indescribably bad and extremel...
                                ...                        
146685    look this game is fun and great but do you kno...
146045    there needs to be balancing for lobby making b...
50043     the game has become full of lag, although befo...
43224     i always heard of the pc version but now it ca...
87865     i like the graphics where i can adjust it. the...
Name: content, Length: 5000, dtype: object

In [ ]:
with open('readme_test.txt', 'w') as f:
    for line in part_20:
        f.write(line)
        f.write('\n')

### Reviews Labeling

In [ ]:
nlp = spacy.blank("en") #load new spacy model
db = DocBin()

In [ ]:
f = open('/content/drive/MyDrive/Tesis/data_training/annotate_all.json')
TRAIN_DATA = json.load(f)
TRAIN_DATA

{'classes': ['GAMEPLAY', 'ACTOR', 'VISUAL', 'AUDIO'],
 'annotations': [["super infair matching system and obvious gap between character's powers\r",
   {'entities': [[53, 62, 'ACTOR']]}],
  ['it is designed using the unreal engine, a video game graphics developer studio, and is known for its realistic graphics, high contrast, and vibrant frames. however, free fire has more arcade-style gameplay and a cartoonish approach towards the graphics quality and visuals in-game\r',
   {'entities': [[111, 119, 'VISUAL'],
     [197, 205, 'GAMEPLAY'],
     [244, 252, 'VISUAL'],
     [265, 272, 'VISUAL']]}],
  ["i love this game don't get me wrong. since the new update, i've been experiencing the same issue as many other players. the game crashes as soon as i open it, it crashes in matchmaking for classique and ranked. made me miss a ban in ranked and has affected my credit score. i find this issue pretty darn annoying as well as everyone.\r",
   {'entities': [[173, 184, 'GAMEPLAY']]}],
  ["it's goo

proses training model, untuk mendapatkan akurasi dari model yang dibentuk

In [ ]:
! python -m spacy init config config.cfg --lang id --pipeline ner --optimize efficiency --force

2023-04-16 23:31:50.914568: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: id
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
texts = [text for text, annot in TRAIN_DATA['annotations']]
annotations = [annot for text, annot in TRAIN_DATA['annotations']]

# add all data to the DocBin object
for text, annot in tqdm(zip(texts, annotations)): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

326it [00:00, 1344.43it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [ ]:
# save all data to disk
db.to_disk("./all_data.spacy")

In [ ]:
import timeit
import numpy as np
from sklearn.model_selection import KFold

# load all data from disk
db = DocBin().from_disk("./all_data.spacy")
docs = list(db.get_docs(nlp.vocab))

# initialize k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True)


# # iterate over k-fold cross-validation splits
# for fold, (train_idx, test_idx) in enumerate(kfold.split(docs)):
#     # create a DocBin object for training data and testing data
#     db_train = DocBin()
#     db_test = DocBin()

#     # iterate over training data for this fold
#     for doc in [docs[i] for i in train_idx]:
#         db_train.add(doc)

#     # iterate over testing data for this fold
#     for doc in [docs[i] for i in test_idx]:
#         db_test.add(doc)

#     # save training and testing data to disk
#     db_train.to_disk(f"./training_data_fold{fold}.spacy")
#     db_test.to_disk(f"./testing_data_fold{fold}.spacy")




In [ ]:
for fold, (train_idx, test_idx) in enumerate(kfold.split(docs)):
    # create a DocBin object for training data and testing data
    db_train = DocBin()
    db_test = DocBin()

    # iterate over training data for this fold
    for doc in [docs[i] for i in train_idx]:
        db_train.add(doc)

    # iterate over testing data for this fold
    for doc in [docs[i] for i in test_idx]:
        db_test.add(doc)

    # save training and testing data to disk
    db_train.to_disk(f"./training_data_fold{fold}.spacy")
    db_test.to_disk(f"./testing_data_fold{fold}.spacy")

    # train the model on this fold
    start = timeit.default_timer()
    ! python -m spacy train config.cfg --output ./kfold{fold} --paths.train ./training_data_fold{fold}.spacy --paths.dev ./training_data_fold{fold}.spacy 
    end = timeit.default_timer()
    print(f"Training time for fold {fold+1}: {end-start:.2f} seconds")

    # evaluate the model on this fold
    result = ! python -m spacy evaluate ./kfold{fold}/model-best/ ./testing_data_fold{fold}.spacy
    print(f"Results for fold {fold+1}:")
    print(result)

2023-04-16 23:32:09.598330: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: kfold0
ℹ Saving to output directory: kfold0
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-04-16 23:32:17,285] [INFO] Set up nlp object from config
[2023-04-16 23:32:17,297] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-04-16 23:32:17,302] [INFO] Created vocabulary
[2023-04-16 23:32:17,303] [INFO] Finished initializing nlp object
[2023-04-16 23:32:17,786] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     14.78    0.00    0.00    0.00    0.00
  1     200         

In [ ]:
print(fold)

4


In [ ]:
# df_label.drop(df_label[4776:4779].index, inplace=True)

In [ ]:
# df_review

In [ ]:
nlp_ner = spacy.load("/content/kfold4/model-best")

In [ ]:
def mydocString(strs):
    """
    Parameters
    ----------
    numbers : list of str
        List of str
    """

    return strs

a = []
for i in tqdm(df_dataset['content']):
    doc =  nlp_ner(mydocString(i))
    a.append(doc.ents)


100%|██████████| 180960/180960 [21:16<00:00, 141.71it/s]


In [ ]:
print(a)

[(controls smooth, graphics), (), (matchmaking ., role), (), (), (matchmaking, the controls), (characters,), (), (gameplay.,), (graphics,), (), (matchmaking,), (), (), (graphics, graphics), (matchmaking, role), (1 hero,), (match making,), (graphics,), (), (), (pay to win,), (graphics,), (), (), (graphics, gameplay.), (), (), (), (matchmaking., matchmaking), (), (pay to win, skins), (), (), (), (ui,), (matchmaking, experience), (pay-to-win, matchmaking), (), (graphics,), (gameplay,), (), (matchmaking,), (), (skins,), (hero,), (graphics, characters, character), (), (match making, character), (), (characters, gameplay, role., matchmaking.), (match making,), (skins, the skin, sound, voice), (), (), (character,), (skins, ui), (google play,), (), (match making,), (matchmaking,), (), (), (matchmaking,), (matchmaking,), (graphics,, match making), (), (match system.,), (game is not good,), (), (match making,), (gameplay, gameplay), (graphics, gameplay, matchmaking, matchmaking), (), (matchmakin

In [ ]:
print('jumlah data berdasarkan aspect kualitas mobile gim: ',len(a))

jumlah data berdasarkan aspect kualitas mobile gim:  180960


In [ ]:
# spacy.displacy.render(a, style="ent", jupyter=True)

In [ ]:
df_dataset['aspect_keyword'] = a
# df.insert(2, "aspect_key", aspect_terms, True)

In [ ]:
df_dataset

,userName,content,score,at,app_name,aspect_keyword
0,Oklahoma Hound,i like this game alot. game controls smooth ga...,3,2023-02-28 23:52:36,League of Legends: Wild Rift,"((controls, smooth), (graphics))"
1,Bryan Bush,for casual players: the most fundamental flaw ...,3,2023-02-07 03:21:16,League of Legends: Wild Rift,()
2,Vincent Agustin,this game is better than other mobile games bu...,3,2023-03-01 16:45:36,League of Legends: Wild Rift,"((matchmaking, .), (role))"
3,Andre Koh,the game is fun and all when you get to play i...,3,2023-01-25 11:28:18,League of Legends: Wild Rift,()
4,Lee Stefan,only exclusive to this game in all games acros...,3,2023-01-25 10:12:39,League of Legends: Wild Rift,()
...,...,...,...,...,...,...
180955,Mayank Gala,dont install matchmaking is worst,1,2020-06-29 08:32:51,Heroes Evolved,"((matchmaking),)"
180956,Johny Nabam,your matchmaking sucks..,2,2020-09-16 15:34:42,Heroes Evolved,"((matchmaking),)"
180957,A Google user,the game is dead,1,2020-04-16 23:27:24,Heroes Evolved,()
180958,noctix nefarioux,lost account,1,2021-09-27 17:58:51,Heroes Evolved,()


hitung setiap elemennya

In [ ]:
df_dataset['aspect_entity'] = df_dataset['aspect_keyword'].map(lambda i: [ent.label_ for ent in i])


In [ ]:
df_dataset

,userName,content,score,at,app_name,aspect_keyword,aspect_entity
0,Oklahoma Hound,i like this game alot. game controls smooth ga...,3,2023-02-28 23:52:36,League of Legends: Wild Rift,"((controls, smooth), (graphics))","[GAMEPLAY, VISUAL]"
1,Bryan Bush,for casual players: the most fundamental flaw ...,3,2023-02-07 03:21:16,League of Legends: Wild Rift,(),[]
2,Vincent Agustin,this game is better than other mobile games bu...,3,2023-03-01 16:45:36,League of Legends: Wild Rift,"((matchmaking, .), (role))","[GAMEPLAY, GAMEPLAY]"
3,Andre Koh,the game is fun and all when you get to play i...,3,2023-01-25 11:28:18,League of Legends: Wild Rift,(),[]
4,Lee Stefan,only exclusive to this game in all games acros...,3,2023-01-25 10:12:39,League of Legends: Wild Rift,(),[]
...,...,...,...,...,...,...,...
180955,Mayank Gala,dont install matchmaking is worst,1,2020-06-29 08:32:51,Heroes Evolved,"((matchmaking),)",[GAMEPLAY]
180956,Johny Nabam,your matchmaking sucks..,2,2020-09-16 15:34:42,Heroes Evolved,"((matchmaking),)",[GAMEPLAY]
180957,A Google user,the game is dead,1,2020-04-16 23:27:24,Heroes Evolved,(),[]
180958,noctix nefarioux,lost account,1,2021-09-27 17:58:51,Heroes Evolved,(),[]


In [ ]:
print('jumlah data aspect actor: ',len(df_dataset[df_dataset['aspect_entity'].map(lambda x: 'ACTOR' in x)]))

jumlah data aspect actor:  24636


In [ ]:
print('jumlah data aspect gameplay: ',len(df_dataset[df_dataset['aspect_entity'].map(lambda x: 'GAMEPLAY' in x)]))

jumlah data aspect gameplay:  28170


In [ ]:
print('jumlah data aspect audio: ',len(df_dataset[df_dataset['aspect_entity'].map(lambda x: 'AUDIO' in x)]))

jumlah data aspect audio:  1905


In [ ]:
print('jumlah data aspect visual: ',len(df_dataset[df_dataset['aspect_entity'].map(lambda x: 'VISUAL' in x)]))

jumlah data aspect visual:  18119


In [ ]:
xy = df_dataset[df_dataset['aspect_entity'].str.len() != 0].reset_index(drop=True)
xy

,userName,content,score,at,app_name,aspect_keyword,aspect_entity
0,Oklahoma Hound,i like this game alot. game controls smooth ga...,3,2023-02-28 23:52:36,League of Legends: Wild Rift,"((controls, smooth), (graphics))","[GAMEPLAY, VISUAL]"
1,Vincent Agustin,this game is better than other mobile games bu...,3,2023-03-01 16:45:36,League of Legends: Wild Rift,"((matchmaking, .), (role))","[GAMEPLAY, GAMEPLAY]"
2,Perserk Von Fallen,"game itself is great, matchmaking is pretty te...",2,2023-01-13 09:36:44,League of Legends: Wild Rift,"((matchmaking), (the, controls))","[GAMEPLAY, GAMEPLAY]"
3,T Boyd,i'm beyond hooked. this game has so much strat...,5,2023-02-13 17:17:29,League of Legends: Wild Rift,"((characters),)",[ACTOR]
4,X T R,i've been playing this game since it got relea...,5,2023-01-22 00:43:59,League of Legends: Wild Rift,"((gameplay, .),)",[GAMEPLAY]
...,...,...,...,...,...,...,...
59650,Jan Emerson Acosta,the game is good overall but this past few day...,1,2021-02-07 03:07:47,Heroes Evolved,"((after, hero),)",[ACTOR]
59651,kawviii san,great graphics while maintaining the 5v5 and 3...,3,2022-07-09 03:44:27,Heroes Evolved,"((graphics), (amazing, character))","[VISUAL, ACTOR]"
59652,Mayank Gala,dont install matchmaking is worst,1,2020-06-29 08:32:51,Heroes Evolved,"((matchmaking),)",[GAMEPLAY]
59653,Johny Nabam,your matchmaking sucks..,2,2020-09-16 15:34:42,Heroes Evolved,"((matchmaking),)",[GAMEPLAY]


In [ ]:
xy['aspect_entity'] = xy['aspect_entity'].map(lambda x: set(x))
xy

,userName,content,score,at,app_name,aspect_keyword,aspect_entity
0,Oklahoma Hound,i like this game alot. game controls smooth ga...,3,2023-02-28 23:52:36,League of Legends: Wild Rift,"((controls, smooth), (graphics))","{GAMEPLAY, VISUAL}"
1,Vincent Agustin,this game is better than other mobile games bu...,3,2023-03-01 16:45:36,League of Legends: Wild Rift,"((matchmaking, .), (role))",{GAMEPLAY}
2,Perserk Von Fallen,"game itself is great, matchmaking is pretty te...",2,2023-01-13 09:36:44,League of Legends: Wild Rift,"((matchmaking), (the, controls))",{GAMEPLAY}
3,T Boyd,i'm beyond hooked. this game has so much strat...,5,2023-02-13 17:17:29,League of Legends: Wild Rift,"((characters),)",{ACTOR}
4,X T R,i've been playing this game since it got relea...,5,2023-01-22 00:43:59,League of Legends: Wild Rift,"((gameplay, .),)",{GAMEPLAY}
...,...,...,...,...,...,...,...
59650,Jan Emerson Acosta,the game is good overall but this past few day...,1,2021-02-07 03:07:47,Heroes Evolved,"((after, hero),)",{ACTOR}
59651,kawviii san,great graphics while maintaining the 5v5 and 3...,3,2022-07-09 03:44:27,Heroes Evolved,"((graphics), (amazing, character))","{ACTOR, VISUAL}"
59652,Mayank Gala,dont install matchmaking is worst,1,2020-06-29 08:32:51,Heroes Evolved,"((matchmaking),)",{GAMEPLAY}
59653,Johny Nabam,your matchmaking sucks..,2,2020-09-16 15:34:42,Heroes Evolved,"((matchmaking),)",{GAMEPLAY}


In [ ]:
data_labeled = xy[['userName','content','score','at','aspect_entity','app_name']].copy()
data_labeled

,userName,content,score,at,aspect_entity,app_name
0,Oklahoma Hound,i like this game alot. game controls smooth ga...,3,2023-02-28 23:52:36,"{GAMEPLAY, VISUAL}",League of Legends: Wild Rift
1,Vincent Agustin,this game is better than other mobile games bu...,3,2023-03-01 16:45:36,{GAMEPLAY},League of Legends: Wild Rift
2,Perserk Von Fallen,"game itself is great, matchmaking is pretty te...",2,2023-01-13 09:36:44,{GAMEPLAY},League of Legends: Wild Rift
3,T Boyd,i'm beyond hooked. this game has so much strat...,5,2023-02-13 17:17:29,{ACTOR},League of Legends: Wild Rift
4,X T R,i've been playing this game since it got relea...,5,2023-01-22 00:43:59,{GAMEPLAY},League of Legends: Wild Rift
...,...,...,...,...,...,...
59650,Jan Emerson Acosta,the game is good overall but this past few day...,1,2021-02-07 03:07:47,{ACTOR},Heroes Evolved
59651,kawviii san,great graphics while maintaining the 5v5 and 3...,3,2022-07-09 03:44:27,"{ACTOR, VISUAL}",Heroes Evolved
59652,Mayank Gala,dont install matchmaking is worst,1,2020-06-29 08:32:51,{GAMEPLAY},Heroes Evolved
59653,Johny Nabam,your matchmaking sucks..,2,2020-09-16 15:34:42,{GAMEPLAY},Heroes Evolved


In [ ]:
#  type(data_labeled['aspect_keyword'][0][0])

In [ ]:
# data_labeled['aspect_keyword'] = data_labeled['aspect_keyword'].map(lambda x: [list(y) for y in x])

In [ ]:
#  type(data_labeled['aspect_keyword'][0][0][0])

In [ ]:
# data_labeled['aspect_keyword'] = data_labeled['aspect_keyword'].map(lambda x: [list([str(z) for z in y]) for y in x])

In [ ]:
#  type(data_labeled['aspect_keyword'][0][0][0])

In [ ]:
df_actor_label = data_labeled[data_labeled['aspect_entity'].map(lambda x: 'ACTOR' in x)]
df_actor_label

,userName,content,score,at,aspect_entity,app_name
3,T Boyd,i'm beyond hooked. this game has so much strat...,5,2023-02-13 17:17:29,{ACTOR},League of Legends: Wild Rift
9,ManAC,"the price of champions is cheap,i don't want t...",5,2023-02-02 23:20:10,{ACTOR},League of Legends: Wild Rift
16,Christian Tenjo,"amazing game, i picked this up a while back an...",5,2023-02-09 23:50:33,"{GAMEPLAY, ACTOR}",League of Legends: Wild Rift
23,2bUnique4u,"it's a good game, the length of match means th...",4,2023-01-24 03:52:00,{ACTOR},League of Legends: Wild Rift
24,Hope Briones,your update system sucks. even though i got go...,1,2023-01-31 06:40:30,{ACTOR},League of Legends: Wild Rift
...,...,...,...,...,...,...
59647,A Google user,save yourself the trouble and ignore/uninstall...,1,2020-01-30 19:40:29,"{GAMEPLAY, ACTOR}",Heroes Evolved
59649,Amon X,"i love the game, great graphics and heroes, ...",5,2020-09-15 20:05:46,"{ACTOR, VISUAL}",Heroes Evolved
59650,Jan Emerson Acosta,the game is good overall but this past few day...,1,2021-02-07 03:07:47,{ACTOR},Heroes Evolved
59651,kawviii san,great graphics while maintaining the 5v5 and 3...,3,2022-07-09 03:44:27,"{ACTOR, VISUAL}",Heroes Evolved


In [ ]:
df_gameplay_label = data_labeled[data_labeled['aspect_entity'].map(lambda x: 'GAMEPLAY' in x)]
df_gameplay_label

,userName,content,score,at,aspect_entity,app_name
0,Oklahoma Hound,i like this game alot. game controls smooth ga...,3,2023-02-28 23:52:36,"{GAMEPLAY, VISUAL}",League of Legends: Wild Rift
1,Vincent Agustin,this game is better than other mobile games bu...,3,2023-03-01 16:45:36,{GAMEPLAY},League of Legends: Wild Rift
2,Perserk Von Fallen,"game itself is great, matchmaking is pretty te...",2,2023-01-13 09:36:44,{GAMEPLAY},League of Legends: Wild Rift
4,X T R,i've been playing this game since it got relea...,5,2023-01-22 00:43:59,{GAMEPLAY},League of Legends: Wild Rift
6,ɪɴsɨɢʜᴛ ᴀᴠᴀɴᴛ-ɢᴀʀᴅᴇ,i like this game. i really do. the only proble...,4,2023-01-12 08:59:14,{GAMEPLAY},League of Legends: Wild Rift
...,...,...,...,...,...,...
59645,Harish Mayekar,superb gameplay if you have lot's of spare tim...,3,2020-09-26 15:45:31,{GAMEPLAY},Heroes Evolved
59646,A Google user,the game needs a jungle update and a matchmaki...,3,2020-04-20 11:38:48,{GAMEPLAY},Heroes Evolved
59647,A Google user,save yourself the trouble and ignore/uninstall...,1,2020-01-30 19:40:29,"{GAMEPLAY, ACTOR}",Heroes Evolved
59652,Mayank Gala,dont install matchmaking is worst,1,2020-06-29 08:32:51,{GAMEPLAY},Heroes Evolved


In [ ]:
df_visual_label = data_labeled[data_labeled['aspect_entity'].map(lambda x: 'VISUAL' in x)]
df_visual_label

,userName,content,score,at,aspect_entity,app_name
0,Oklahoma Hound,i like this game alot. game controls smooth ga...,3,2023-02-28 23:52:36,"{GAMEPLAY, VISUAL}",League of Legends: Wild Rift
5,Lay Low,"overall the game gets 4 out of 5 from me,it's ...",4,2023-02-15 08:40:20,{VISUAL},League of Legends: Wild Rift
7,Stefan Mironov,better than pc league. they put so much love i...,5,2023-01-12 05:53:45,{VISUAL},League of Legends: Wild Rift
11,ivan brinosa,all throughout a great game with a very big po...,2,2023-02-09 06:01:32,{VISUAL},League of Legends: Wild Rift
13,Red_ Diamond,perfect i have played games on mobile but this...,3,2023-02-28 19:56:06,{VISUAL},League of Legends: Wild Rift
...,...,...,...,...,...,...
59639,A Google user,for me best moba game ....i am totally addicte...,5,2020-01-06 05:08:12,{VISUAL},Heroes Evolved
59643,たかスクナミ,very good graphics and smooth gameplay.keep up...,4,2021-09-17 06:42:13,{VISUAL},Heroes Evolved
59648,Kalvin Gani,the graphic good control also best game,5,2022-03-30 01:05:22,{VISUAL},Heroes Evolved
59649,Amon X,"i love the game, great graphics and heroes, ...",5,2020-09-15 20:05:46,"{ACTOR, VISUAL}",Heroes Evolved


In [ ]:
df_audio_label = data_labeled[data_labeled['aspect_entity'].map(lambda x: 'AUDIO' in x)]
df_audio_label

,userName,content,score,at,aspect_entity,app_name
29,Drei Aswell,your customer service is just a hassle to go t...,5,2023-01-14 00:21:43,"{ACTOR, AUDIO}",League of Legends: Wild Rift
55,Jeremy Maza,i would love to raye this a 5 stars i love the...,1,2023-01-17 15:40:12,{AUDIO},League of Legends: Wild Rift
56,Aaron Ryowo,ml is somehow better performance wise. at the ...,1,2023-02-25 01:35:11,"{GAMEPLAY, AUDIO}",League of Legends: Wild Rift
93,Ghost,after your latest update. i played once and af...,1,2023-02-03 07:21:35,{AUDIO},League of Legends: Wild Rift
153,Rod Stephen Fabul,i like this game a lot i played it on pc and m...,4,2023-02-11 08:50:27,"{ACTOR, AUDIO}",League of Legends: Wild Rift
...,...,...,...,...,...,...
59406,Mark Arriane Martinez,change game voice 😀.,4,2021-01-31 16:46:48,{AUDIO},Heroes Evolved
59417,Rohit Thapa,we want voice chat....... please,2,2020-07-02 15:30:17,{AUDIO},Heroes Evolved
59446,A Google user,i have rate 5*,5,2020-01-03 07:06:07,{AUDIO},Heroes Evolved
59523,wien sulistyono,back in voice chat,3,2020-10-20 10:49:59,{AUDIO},Heroes Evolved


In [ ]:
# df_review.to_csv(r'./export_dataframe.csv', index=False, header=True)
data_labeled.to_pickle('/content/drive/MyDrive/Tesis/saved_data/data_with_label_up2.pkl', compression='infer', protocol=5)
